<a href="https://colab.research.google.com/github/MateoTokic/docTR/blob/main/python_doctr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Doctr

In [ ]:
!pip install python-doctr[tf]  # This installs Doctr with torch backend support


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.5/299.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 25.7 MB/s eta 0:00:00
   ━━

In [ ]:
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
import re


from google.colab import drive
drive.mount('/content/drive')

def mount_drive():
    from google.colab import drive
    drive.mount('/content/drive')

# Call the function to mount
mount_drive()

photo_folder= '/content/drive/MyDrive/proizvodi'

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Read the image
image_path_ingredients = '/content/drive/MyDrive/proizvodi/kukuruz_sastojci.jpg'
image_path_nutri_values = '/content/drive/MyDrive/proizvodi/kukuruz_tablica.jpg'

doc_ingredients = DocumentFile.from_images(image_path_ingredients)
doc_nutri_values = DocumentFile.from_images(image_path_nutri_values)

In [ ]:
# Initialize the OCR predictor
model = ocr_predictor(pretrained=True)


# Perform OCR on the document
result_ingredients = model(doc_ingredients)
result_nutri_values = model(doc_nutri_values)

  0%|          | 0/60957071 [00:00<?, ?it/s]

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


  0%|          | 0/58758994 [00:00<?, ?it/s]

# Extract ingredients function

In [ ]:
import re

# Function to extract ingredients after 'sastoj' or 'Sastoj' and before '.'
def extract_ingredients(page):
    found_sastoj = False  # Track if we found 'sastoj' or 'Sastoj'
    extracted_ingredients = []  # Store extracted ingredients
    current_ingredient = ""  # Track ongoing ingredient name

    # Loop over blocks, lines, and words
    for block in page['blocks']:
        for line in block['lines']:
            for word in line['words']:
                word_value = word['value']
                confidence = word['confidence']

                # Debugging: print recognized words and confidence
                print(f"Word: {word_value}, Confidence: {confidence}")

                # Check if 'sastoj' or 'Sastoj' is in the word
                if re.search(r'[Ss]astoj', word_value):
                    print(f"Found 'sastoj' or 'Sastoj' at word: {word_value}")
                    found_sastoj = True
                    continue  # Skip the match itself and start extracting after

                # If 'sastoj' or 'Sastoj' was found, start extracting words
                if found_sastoj:
                    # Stop extraction at the first period ('.')
                    if '.' in word_value:
                        print(f"Found '.' in word: {word_value}, stopping extraction.")
                        if current_ingredient:
                            extracted_ingredients.append(current_ingredient.strip())  # Add final ingredient
                        current_ingredient = ""  # Reset for next line
                        continue  # Don't stop extraction, allow for multiple lines

                    # Filter out phrases that are likely not ingredients (e.g., instructions)
                    if any(phrase in word_value.lower() for phrase in ['upotrijebiti', 'cuvajte', 'otvaranja', 'hladnjaku', 'konzumirati', 'sati']):
                        print(f"Skipping non-ingredient phrase: {word_value}")
                        continue

                    # If the word contains a comma, connect it to the previous word
                    if ',' in word_value:
                        current_ingredient += f" {word_value}"  # Append current word (with comma)
                        extracted_ingredients.append(current_ingredient.strip())
                        current_ingredient = ""  # Reset after extracting full ingredient
                    else:
                        # If the word doesn't contain a comma, continue building the ingredient name
                        current_ingredient += f" {word_value}"

                    # Debugging: Print the current ingredient being built
                    print(f"Building ingredient: {current_ingredient.strip()}")

    # Final cleanup: Append any leftover ingredient
    if current_ingredient:
        extracted_ingredients.append(current_ingredient.strip())

    # If no ingredients found, return a message
    if not extracted_ingredients:
        print("No ingredients found after 'sastoj' or 'Sastoj' and before '.'")
    else:
        print(f"Extracted ingredients: {extracted_ingredients}")

    return extracted_ingredients



# Example usage
for page in result_ingredients.pages:
    page_content = page.export()  # Export the page content to a dictionary
    extracted_ingredients = extract_ingredients(page_content)

# Display the extracted ingredients
print("Extracted ingredients after 'sastoj' or 'Sastoj':")
print(", ".join(extracted_ingredients))


Word: Kukuruz, Confidence: 0.9994193911552429
Word: secerac, Confidence: 0.8545039296150208
Word: U, Confidence: 0.9990220069885254
Word: vakuumu, Confidence: 0.9974378943443298
Word: Sastojci:, Confidence: 0.9505699872970581
Found 'sastoj' or 'Sastoj' at word: Sastojci:
Word: kukuruz,, Confidence: 0.8514907956123352
Building ingredient: 
Word: voda,, Confidence: 0.9986028671264648
Building ingredient: 
Word: secer,, Confidence: 0.8427552580833435
Building ingredient: 
Word: kuhinjska, Confidence: 0.46208834648132324
Building ingredient: kuhinjska
Word: sol/so, Confidence: 0.9936696290969849
Building ingredient: kuhinjska sol/so
Word: Najbolje, Confidence: 0.9953492879867554
Building ingredient: kuhinjska sol/so Najbolje
Word: upotrijebiti, Confidence: 0.9788751006126404
Skipping non-ingredient phrase: upotrijebiti
Word: do:, Confidence: 0.999636709690094
Building ingredient: kuhinjska sol/so Najbolje do:
Word: vidina, Confidence: 0.9837634563446045
Building ingredient: kuhinjska sol/s

# Extract nutrition values function

In [ ]:
# Function to extract nutrition labels and their values
def extract_nutrition_values(page):
    nutrition_data = {}
    current_label = None

    # Define common nutrition labels (both Croatian and English)
    nutrition_labels = [
        r'Energ', r'Masti', r'Ugl', r'Bjel',
        r'Prot', r'Vlakna', r'Sol',
        r'zasic', r'nezasi',
        r'sece'
    ]

    # Combine all labels into one regex pattern
    label_pattern = '|'.join(nutrition_labels)

    # First, print all recognized words for debugging
    # print("All recognized words:")
    for block in page['blocks']:
        for line in block['lines']:
            for word in line['words']:
                word_value = word['value']
                confidence = word['confidence']
                # print(f"Word: {word_value}, Confidence: {confidence}")

    # Extract recognized words and check if they match any nutrition label or numeric value
    # print("\nExtracted Nutrition Labels and Values:")
    for block in page['blocks']:
        for line in block['lines']:
            for word in line['words']:
                word_value = word['value']
                confidence = word['confidence']

                # Clean the word value from special characters and whitespaces
                word_value = word_value.strip().replace(',', '.')

                # Check if the word is a nutrition label
                if re.search(label_pattern, word_value):
                    current_label = word_value
                    nutrition_data[current_label] = None
                    # print(f"Found label: {word_value}")

                # If a label was found earlier, look for the corresponding value
                elif current_label and re.search(r'\d+[.,]?\d*\s*[g|mg|kcal|kJ]', word_value):
                    # Assign the value to the corresponding label
                    nutrition_data[current_label] = word_value
                    # print(f"Value for {current_label}: {word_value}")
                    current_label = None  # Reset after finding the value

    return nutrition_data

# Example to test the function
for page in result_nutri_values.pages:
    page_content = page.export()  # Export the page content to a dictionary
    nutrition_values = extract_nutrition_values(page_content)
    print("Extracted Nutrition Data:", nutrition_values)

Extracted Nutrition Data: {'Energetska': '453kJ/107', 'Masti': '15g', 'odkojih:zasicene': '0.2g', 'Uopondati/Uglenihdrati': '19g', 'secer': '3.4g', 'Vlakna': '3.3g', 'Selancevine/Proteint': '2.8g', 'Sol/So': '0.45g'}


In [ ]:
!pip install openai==0.28



In [ ]:
import openai
import os

# Set up OpenAI API Key
openai.api_key = ''


In [ ]:
def get_nutritional_advice(nutrition_values, extracted_ingredients):
    # Prepare the prompt using the extracted data
    prompt = f"""
    Here are the nutritional values of a product:
    {', '.join([f"{label}: {value}" for label, value in nutrition_values.items() if value])}

    Ingredients of the product are:
    {', '.join(extracted_ingredients)}

    Please provide nutritional advice based on the above information. Include insights like daily recommended intake, potential health benefits, and warnings if applicable.
    """

    # Send the prompt to the OpenAI API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use gpt-3.5-turbo model
        messages=[{"role": "system", "content": "You are a nutrition expert."},
                  {"role": "user", "content": prompt}],
        max_tokens=300,  # Adjust based on how long you expect the response to be
        temperature=0.7  # Adjust temperature to control creativity of response
    )

    # Extract the response text
    advice = response['choices'][0]['message']['content']
    return advice



# Get nutritional advice
advice = get_nutritional_advice(nutrition_values, extracted_ingredients)
print(advice)

Based on the provided nutritional values and ingredients, here is some advice:

1. **Energy (Energetska):** The product provides 453 kJ of energy. It's important to consider your overall energy intake throughout the day to maintain a healthy weight and energy balance.

2. **Fats (Masti):** The product contains 15g of fats, with only 0.2g being saturated fats. It's good to limit saturated fats in the diet as they can increase cholesterol levels and risk of heart disease. Aim to choose products with lower saturated fat content.

3. **Carbohydrates (Uopondati/Uglenihdrati):** The product contains 19g of carbohydrates, including 3.4g of sugar. It's important to be mindful of added sugar intake as excessive sugar consumption can contribute to weight gain and various health issues. Opt for products with lower sugar content.

4. **Fiber (Vlakna):** The product provides 3.3g of fiber, which is beneficial for digestive health, helps with satiety, and can aid in managing blood sugar levels. It's